<table align="left" style="border-style: hidden" class="table"> <tr><td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon_sp22_ugarte.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Fall 2022</h4><p>Ani Adhikari</p>CC BY-NC-SA 4.0</div></td></tr></table><!-- not in pdf -->

This content is protected and may not be shared, uploaded, or distributed.

In [ ]:
from datascience import *
from prob140 import *
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

# Lab 4: Multinomial Correlations

Many questions in data science involve populations that can be partitioned into categories of individuals. For example, in Data 8 you classified individuals into one of two classes. Data 100 examines logistic regression as a different approach to binary classification. These methods can be extended to multiple categories.

At a more fundamental level, it is important to understand how a random sample from a categorical population reflects the population. As you know, the [multinomial](http://prob140.org/textbook/content/Chapter_06/03_Multinomial_Distribution.html#id1) is the joint distribution of the counts of the different categories in a random sample of a fixed size. 

More precisely, suppose the population has $k$ categories of individuals in proportions $p_1, p_2, \ldots, p_k$ with $\sum_{i=1}^k p_i = 1$. For $1 \le i \le k$ let $N_i$ be the number of elements in Category $i$ in a sample of size $n$ drawn at random with replacement from the population. Then the joint distribution of of $N_1, N_2, \ldots, N_k$ is multinomial with parameters $n$ and $p_1, p_2, \ldots, p_k$.

The counts $N_1, N_2, \ldots, N_k$ are dependent since $\sum_{i=1}^k N_i = n$ which is a constant. In this lab you will study the dependence between pairs $N_i$ and $N_j$ and develop a way to predict $N_j$ based on the value of $N_i$. You will see that the method is closely connected to the linear regression that you studied in Data 8.

What you will learn in this lab:

- How to simulate multinomial counts in Python
- How to find the covariance of two multinomial counts and hence the correlation between them
- How to predict one multinomial count based on another, and how this is connected to linear regression

A few computational preliminaries will help you get started.

## Part 1: Preliminaries

### 1a) Simulating a Multinomial
`np.random.multinomial(n, p_array)` simulates `n` draws at random with replacement from the categorical distribution `p_array` and returns an array of the observed counts in all the categories. 

You have seen a version of this in Data 8. The `datascience` [function](https://inferentialthinking.com/chapters/11/2/Multiple_Categories.html#comparison-with-panels-selected-at-random) `sample_proportions` is just `np.random.multinomial` with the output converted to proportions.

Run the cell below to draw 10 times from a population in which 20% are in Category A, 70% in Category B, and 10% in Category C. The output array contains the observed counts in Categories A, B, and C. 

Confirm that it makes sense, by looking at the total count and the count in each category. Then run the cell a few times to see how the observations change.

In [ ]:
num_draws = 10
population_proportions = [0.2, 0.7, 0.1] # make_array(0.2, 0.7, 0.1) is also fine
np.random.multinomial(num_draws, population_proportions)

### 1b) Adding Rows to a Table
You know that `t.with_columns` can be used to add a column or columns to a table `t`. Sometimes it is useful to grow a table by adding rows. Start by creating the table with just the column labels and no rows, as follows.

In [ ]:
t = Table(['Column A', 'Column B', 'Column C'])
t

Now `append` a row to the table. It is important to note that `t.append` mutates (that is, changes) the table `t`. Unlike the usual Table operations, it doesn't just display a temporary copy of `t` that you have to name if you want to save it. Run the two cells below and keep track of what happens to `t`.

In [ ]:
t.append([1, 2, 3])

In [ ]:
# t has been changed
t

To append another row, use `append` again.

In [ ]:
t.append([4, 5, 6])

In [ ]:
# t has been changed
t

### 1c) Putting the Two Together
Create a table `simulated_counts` that has column labels `A`, `B`, and `C`. The table should have two rows. Each row should contain the observed counts in Categories A, B, and C in 10 multinomial trials, each of which results in Category A with chance $0.2$, Category B with chance $0.7$, and Category C with chance $0.1$ as in Part **a** above. The trials in each row should be independent of those in the other row.

You should use no more than 3 lines of code.

In [ ]:
# Answer to 1c

...
...
...

## 2. Marginals
Each spin of a Nevada roulette wheel results in the winning color being red with chance $18/38$, black with chance $18/38$, and green with chance $2/38$, independent of all other spins.

A Nevada roulette wheel is spun $380$ times. Let $R$ be the number of times red wins, let $B$ be the number of times black wins, and let $G$ be the number of times green wins.

### 2a) The Distributions
Fill in the blanks below with the names of famous distributions and the relevant parameters.

(i) The joint distribution of $R$, $B$, and $G$ is $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~}$ with parameters $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$.

(ii) $R$ has the $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~(~~~~~~~~~~~~~~~)}$ distribution.

(iii) $B$ has the $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~(~~~~~~~~~~~~~~~)}$ distribution.

(iv) $G$ has the $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~(~~~~~~~~~~~~~~~)}$ distribution.


**Your answer here**

### 2b) Probability Histogram of $R$

Draw the probability histogram of $R$.

In [ ]:
# Answer to 2b

k = np.arange(130, 231)
p_k = ...
dist_R = Table().values(k).probabilities(p_k)
Plot(dist_R, show_ev=True, show_sd=True)
plt.title('Distribution of $R$');

### 2c) Expectations and SDs
Use Part **a** to write numerical expressions for the expectations and SDs of the three random counts. Check that the values of $E(R)$ and $SD(R)$ are consistent with the histogram in Part **b**.

In [ ]:
# Answer to 2c

exp_R = ...  # E(R)
sd_R =  ...  # SD(R)
exp_B = ...
sd_B = ...
exp_G = ...
sd_G = ...
exp_R, sd_R, exp_B, sd_B, exp_G, sd_G

### 2d) Empirical Histogram of $R$
Create a table `simulated` that has three columns labeled `R`, `B`, and `G`. The table should have 10,000 rows. Each row should consist of the observed values of $R$, $B$, and $G$ in one set of 380 spins of the roulette wheel. The spins in each row should be independent of those in the other rows.

Then draw an empirical histogram of the distribution of $R$ and compare it briefly with the histogram in Part **b**. Use as many lines as you need before the last two lines provided.

In [ ]:
# Answer to 2d

...

simulated.hist('R', bins=np.arange(129.5, 231, 1))
plt.title('Empirical Distribution of $R$'); # COMPARE WITH 2b

### 2e) Empirical Mean and SD
Find the mean and SD of the 10,000 simulated values of $R$ and compare them to $E(R)$ and $SD(R)$ as found in Part **c**.

In [ ]:
# Answer to 2e

emp_mean_R = ...
emp_SD_R = ...
emp_mean_R, emp_SD_R  # COMPARE TO E(R) AND SD(R)

## 3. Covariance and Correlation
The multinomial is a joint distribution, so now let's look at the relation between $R$ and the other two variables.

### 3a) Sign ($+$ or $-$) of Correlation
Just based on your [Data 8](https://inferentialthinking.com/chapters/15/1/Correlation.html#the-correlation-coefficient) recollection of properties of correlation, what would you guess as the sign of the correlation between $R$ and $B$, and why?


**Your answer here**

### 3b) [On Paper] Covariance of $R$ and $B$

Find $Cov(R, B)$ using both the methods below. The first is a direct calculation using the most important property of covariance. The second is a consequence of the particular structure of $R$, $B$, and $G$.

**Method 1, using biliniearity:** For each spin $i$, define indicators $X_i$ and $Y_i$ such that $R = \sum_{j=1}^{380} X_j$ and $B = \sum_{k=1}^{380} Y_k$. Then use bilinearity to find $Cov(R, B)$ and check whether its sign is the same as in Part **a**.

[It will help to keep in mind that for $j \neq k$, $X_j$ is independent of $Y_k$. Then think carefully about $Cov(X_j, Y_j)$.]

**Method 2, using the variance of a sum:** Identify the distribution of $R+B$ and hence find $Var(R+B)$. Then use Part **2c** and the formula for the variance of a sum to find $Cov(R, B)$.



### 3c) [On Paper] Correlation ###
The covariance of random variables $X$ and $Y$ has nasty units: the product of the units of $X$ and the units of $Y$. Dividing the covariance by the two SDs results in an important pure number.

Define the *correlation coefficient* between random variables $X$ and $Y$ as

$$
Corr(X, Y) ~ = \frac{Cov(X, Y)}{SD(X)SD(Y)}
$$

and is called *correlation* for short.

Find $Corr(R, B)$.

### 3d) Empirical Correlation
Let `v_1` and `v_2` be numerical arrays of the same length. The expression `np.corrcoef(v_1, v_2)` evaluates to a $2 \times 2$ *correlation matrix* whose $(i, j)$ element is the correlation (as defined in Data 8) between `v_i` and `v_j`. Thus both the diagonal elements are equal to $1$, and both the off-diagonal elements are the correlation between `v_1` and `v_2` just as you would have calculated it in Data 8.

Use **2d** to get the empirical correlation between $R$ and $B$ based on 10,000 simulations, in a correlation matrix `corr_matrix`. Make sure the result is consistent with your answer to Part **c** above.

Make sure the result is consistent with your answer to Part **c** above.

Later in the lab, to access just the off-diagonal element you can use `corr_matrix[0, 1]`.

In [ ]:
# Answer to 3d

corr_matrix = ...
corr_matrix

### 3e) Visualization

Draw the scatter plot of the 10,000 $(R, B)$ points simulated in **2d**, with $R$ on the horizontal axis. Use only one line of code before the two lines provided. Check that the plot looks consistent with the correlation you found in Part **d**.

In [ ]:
# Answer to 3e

...
plt.xlim(130, 230)
plt.ylim(130, 230);

## 4. Conditional Expectation

Clearly, there's a straight line going through the scatter plot in **3e**. Let's try to find a plausible candidate for that line.

### 4a) [On Paper] A Conditional Distribution
Let $0 \le r+b \le 380$. 

Find $P(B = b \mid R = r)$. Use your formula to answer the following.

(i) Identify the conditional distribution of $B$ given $R=r$ as one of the famous ones and provide its parameters.

(ii) Find $E(B \mid R)$ and show that it is a linear function of $R$.

### 4b) Another Visualization
Redraw the scatter plot in **3e** and overlay the line $R \to E(B \mid R)$.

In [ ]:
# Answer to 4b

r = np.arange(130, 231)
exp_B_given_r = ...  # E(B | R=r)
simulated.scatter('R', 'B')
plt.plot(r, exp_B_given_r, color='green', lw=3, label='$E(B \mid R)$')
plt.legend();

Later in the term we'll see that $E(B \mid R)$ is the best among all predictors of $B$ based on $R$, in the sense of least squared error. The result is valid for any two jointly distributed random variables and does not involve observed data.

### 4c) The Regression Line Based on Data
In Data 8, the regression line was calculated as the best straight line for prediction using the data points at hand. In our current setting that would be the least squares line based on the 10,000 simulated points, not based on the underlying distribution of those points like the conditional expectation line you found in Part **b** above.

Let's calculate the Data 8 line based on the empirical averages and SDs in **2e**. [Recall](https://inferentialthinking.com/chapters/15/2/Regression_Line.html#the-equation-of-the-regression-line) the formulas for the slope and intercept of the regression line, and find the slope and intercept using the simulated data in **2d**. 

Remember that in **2e** you already found `emp_mean_R`, the empirical mean of $R$, and `emp_SD_R`, the empirical SD of $R$. In **3d** you found the empirical correlation between $R$ and $B$. You just need a couple of quantities related to $B$, and then you can find the slope and intercept of the regression line based on the 10,000 simulated points.

In [ ]:
# Answer to 4c

emp_mean_B = ...
emp_SD_B = ...
reg_slope = ...
reg_intercept = ...
reg_slope, reg_intercept

The resulting line is an excellent estimate of the underlying conditional expectation line $R \to E(B \mid R)$ that you plotted in **4b**.

## Conclusion

What you have learned in this lab:

- How to simulate multinomial variables in Python
- How probability distributions and properties such as expectation, standard deviation, and correlation are related to empirical distributions and their corresponding properties
- How to find the correlation between two multinomial counts
- That conditional expectation can be thought of as a way to make predictions
- That for predicting one multinomial count based on another, based on a large amount of data, the empirical regression line looks very much like the conditional expectation of the response given the predictor

## Submission Instructions ##

Many assignments throughout the course will have a written portion and a code portion. Please follow the directions below to properly submit both portions.

### Written Portion ###
*  Scan all the pages into a PDF. You can use any scanner or a phone using applications such as CamScanner. Please **DO NOT** simply take pictures using your phone. 
* Please start a new page for each question. If you have already written multiple questions on the same page, you can crop the image in CamScanner or fold your page over (the old-fashioned way). This helps expedite grading.
* It is your responsibility to check that all the work on all the scanned pages is legible.

### Code Portion ###
* Save your notebook using File > Save and Checkpoint.
* Generate a PDF file using File > Download as > PDF via LaTeX. This might take a few seconds and will automatically download a PDF version of this notebook.
    * If you have issues, please make a follow-up post on the general Lab 4 Ed thread.
    
### Submitting ###
* Combine the PDFs from the written and code portions into one PDF.  [Here](https://smallpdf.com/merge-pdf) is a useful tool for doing so. 
* Submit the assignment to Lab 4 on Gradescope. 
* **Make sure to assign each page of your pdf to the correct question.**
* **It is your responsibility to verify that all of your work shows up in your final PDF submission.**

If you have questions about scanning or uploading your work, please post a follow-up to the [Ed thread](https://edstem.org/us/courses/24954/discussion/1695227) on this topic. 